In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
5990,0.0,0.0,-2.050549,-0.199381,-1.552340,0.799793,4.934311,-5.757187
3388,0.0,0.0,-3.548649,-0.257785,0.530645,-1.162405,4.965229,-0.963652
4986,0.0,1.0,0.131979,-0.005741,0.802263,0.713667,5.370491,-4.870585
8373,1.0,1.0,-8.393166,-0.203675,0.756857,0.693579,-1.229956,2.083529
7914,1.0,0.0,-6.197614,-0.306675,-2.550170,-3.234969,-1.798375,-8.208726
...,...,...,...,...,...,...,...,...
6575,0.0,1.0,-0.781897,0.012702,-0.697226,1.619470,-1.950408,-3.599894
1641,1.0,1.0,-9.728499,0.021730,0.650997,1.127904,-6.242906,-1.116641
1202,1.0,0.0,22.160800,-0.124742,4.615669,2.293163,9.242312,10.226844
3010,0.0,1.0,-8.636453,-0.254793,4.776883,-0.887781,-2.343886,3.576812


In [5]:
from string import digits
import re
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
auc=['0']
df=pd.DataFrame([])
for name in os.listdir('./Model/'):
    full_name=name
    model=torch.load(f'./Model/{name}')
    name=name.split('_')[1]
    name=name.replace('.pth','')
    #print('FIRST',name)
    #print(name[:-1])
    if name[-1].isdigit():
        name=name[:-1]
    #print('Second',name)

    dataset = pd.read_csv(f'../data/{name}/{name}.csv')
    continuous_wachter = dataset.drop(columns=['label']).columns
    scaler='Identity'
    if name == 'economic':
        scaler='MinMax'

    dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
    dataset=dataset._df_test
    data_x, data_y=dataset.drop(columns=['label']), dataset['label']
    data_pred= model.predict(data_x)
    model_name.append(full_name)
    print(data_pred.shape)
    print(data_y.shape)
    if len(data_pred.shape)>1:
        data_pred= np.argmax(data_pred, axis=1)

    accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
    precision.append(sklearn.metrics.precision_score(data_y,data_pred))
    recall.append(sklearn.metrics.recall_score(data_y,data_pred))
    auc.append(sklearn.metrics.roc_auc_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df['auc'] =auc
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model

(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TO

,model_name,accuracy,precision,recall,auc
0,0,0,0,0,0
1,Forest_credit.pth,0.8012,0.829208,0.776062,0.802139
2,Forest_economic.pth,0.8076,0.75805,0.810987,0.808018
3,Forest_sanity-3-lin.pth,0.7988,0.808882,0.795632,0.798883
4,Forest_sanity-3-non-add.pth,0.868,0.931343,0.504039,0.745906
5,Forest_sanity-3-non-lin.pth,0.9692,0.976824,0.957912,0.968667
6,Linear_credit.pth,0.7516,0.764372,0.740973,0.751808
7,Linear_economic.pth,0.3616,0.338074,0.477626,0.3744
8,Linear_sanity-3-lin.pth,0.996,0.996899,0.995356,0.996022
9,Linear_sanity-3-non-add.pth,0.9992,1,0.996918,0.998459


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    if st is '':
        st += row['model_name'].split('_')[1].replace('.pth','')
    else:
        st += ' & '+row['model_name'].split('_')[1].replace('.pth','')
    col.append(row['model_name'].split('_')[0])
    name.append(row['model_name'].split('_')[1])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name)
print(col)
print(name)
for i in range(0,len(col)):
    st += col[i] 
    for j in range(0,len(name)):

        #print()
        print(f'{col[i]}_{name[i]}')
        print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

['Forest' 'Linear' 'MLP']
['credit.pth' 'economic.pth' 'sanity-3-lin.pth' 'sanity-3-lin0.pth'
 'sanity-3-non-add.pth' 'sanity-3-non-lin.pth']
Forest_credit.pth
0.7984
Forest_credit.pth
0.7996
Forest_credit.pth
0.814
Forest_credit.pth


IndexError: index 0 is out of bounds for axis 0 with size 0